### Strategy-2 BD-3,SD+5

In [1]:
# importing necessary libraries
import pandas as pd
import yfinance as yf
import time
import datetime

In [2]:
#taking start_time
start_time = time.time()

# stocks_list
stocks = ["BHARTIARTL.NS","SBIN.NS","DRREDDY.NS",
          "GSPL.NS","JSWSTEEL.NS","JSWENERGY.BO",
          "RELIANCE.NS","YESBANK.NS","TATAMOTORS.NS",
          "NTPC.NS","SAIL.BO","LAURUSLABS.NS"]

stocks_df = pd.DataFrame()

for stock in stocks:
    # reading stocks data from '2015-01-01' to '2022-12-31'
    df = yf.Ticker(stock).history(interval='1d',start="2015-01-01", end = "2022-12-31")
    df.reset_index(level=0, inplace=True)
    df.drop(['High','Low','Volume','Dividends','Stock Splits'],axis=1, inplace=True)
    df['stock'] = stock
    
    invest  = []
    for i in range(len(df)-1):

        # Check if the opening price is more than the closing price on the same date
        if df.loc[i,'Open'] < df.loc[i,'Close']:


            # Set the buying price to the opening price and selling price to the closing price
            buying_price = df.loc[i,'Open']
            buying_date = df.loc[i,'Date']
            selling_price = df.loc[i,'Close']
            selling_date = df.loc[i,'Date']

            # setting the buying price to min(open,close) of previous 3 rows
            if buying_price >  min(df.loc[max(0,i-1):max(i-3,0):-1, ['Open','Close']].min()):
                buying_price = min(df.loc[max(0,i-1):max(i-3,0):-1, ['Open','Close']].min())
                buying_date = df.loc[df[['Open','Close']].eq(buying_price).any(axis=1), 'Date'].iloc[0]

            # setting the selling price to max(open,close) of next 5 rows
            if selling_price < max(df.loc[i+1:min(i+5,len(df)), ['Open','Close']].max()):
                selling_price = max(df.loc[i+1:min(i+5,len(df)), ['Open','Close']].max())
                selling_date = df[df['Date']>buying_date].loc[df[['Open','Close']].eq(selling_price).any(axis=1), 'Date'].iloc[0]

            return_rate = (selling_price - buying_price)/buying_price*100
            
            # checking the return rate is more than the break_even return and appending in invest
            if return_rate > 5.3:
                invest.append({
                'Buying_date': buying_date,
                'Buying_price': buying_price,
                'Selling_date':selling_date,
                'Selling_price':selling_price,
                'Stock': stock,
                'Return_rate':return_rate
        })
                
    invest_df = pd.DataFrame(invest)
    
    # drop duplicates rows
    invest_df.sort_values(by=['Selling_date','Return_rate'], ascending = [True,False], inplace=True)
    invest_df.drop_duplicates(inplace=True)
    
    # dropping duplicates on the basis of selling_date and buying_date
    invest_df.drop_duplicates(subset=['Selling_date'],inplace=True)
    invest_df.drop_duplicates(subset=['Buying_date'],inplace=True)
    
    # calculating the days_diff between the next buying_date and current row selling_date
    invest_df['Days_diff'] = (invest_df['Buying_date'] - invest_df['Selling_date'].shift(1)).dt.days
    invest_df = invest_df.fillna(1)
    invest_df = invest_df[invest_df['Days_diff']>=0]
    
    stocks_df = pd.concat([stocks_df, invest_df])

end_time = time.time()

print("Execution time:", end_time - start_time, "seconds")
stocks_df

Execution time: 341.8179199695587 seconds


Buying_date  Buying_price              Selling_date  \
0   2015-01-14 00:00:00+05:30    304.850220 2015-01-22 00:00:00+05:30   
6   2015-03-10 00:00:00+05:30    301.544283 2015-03-13 00:00:00+05:30   
10  2015-03-20 00:00:00+05:30    335.220612 2015-03-27 00:00:00+05:30   
13  2015-03-27 00:00:00+05:30    331.914734 2015-04-08 00:00:00+05:30   
19  2015-05-04 00:00:00+05:30    329.358092 2015-05-08 00:00:00+05:30   
..                        ...           ...                       ...   
368 2022-07-26 00:00:00+05:30    483.288788 2022-08-02 00:00:00+05:30   
379 2022-08-29 00:00:00+05:30    549.078358 2022-08-30 00:00:00+05:30   
380 2022-09-26 00:00:00+05:30    494.020782 2022-10-04 00:00:00+05:30   
384 2022-10-12 00:00:00+05:30    497.914246 2022-10-20 00:00:00+05:30   
386 2022-10-25 00:00:00+05:30    452.141052 2022-11-03 00:00:00+05:30   

     Selling_price          Stock  Return_rate  Holding_period  \
0       329.137740  BHARTIARTL.NS     7.967034             8.0   
6       352.852173  BHARTIARTL.NS    17.015043             3.0   
10      353.513427  BHARTIARTL.NS     5.456948             7.0   
13      369.514069  BHARTIARTL.NS    11.328010            12.0   
19      350.824554  BHARTIARTL.NS     6.517667             4.0   
..             ...            ...          ...             ...   
368     526.516219  LAURUSLABS.NS     8.944431             7.0   
379     578.329232  LAURUSLABS.NS     5.327268             1.0   
380     522.522949  LAURUSLABS.NS     5.769427             8.0   
384     528.213341  LAURUSLABS.NS     6.085204             8.0   
386     477.950012  LAURUSLABS.NS     5.708166             9.0   

     Return_rate_per_day  Days_diff  
0               0.995879        1.0  
6               5.671681       42.0  
10              0.779564        7.0  
13              0.944001        0.0  
19              1.629417       21.0  
..                   ...        ...  
368             1.277776        8.0  
379             5.327268       11.0  
380             0.721178       27.0  
384             0.760650        8.0  
386             0.634241        5.0  

[1169 rows x 9 columns]

In [4]:
stocks_df.sort_values(by=['Selling_date','Return_rate'], ascending = [True,False], inplace=True)
stocks_df.drop_duplicates(inplace=True)

stocks_df

Buying_date  Buying_price              Selling_date  \
0   2015-01-01 00:00:00+05:30    489.054225 2015-01-05 00:00:00+05:30   
0   2015-01-05 00:00:00+05:30    105.001072 2015-01-09 00:00:00+05:30   
0   2015-01-06 00:00:00+05:30     84.216072 2015-01-15 00:00:00+05:30   
3   2015-01-07 00:00:00+05:30    488.411377 2015-01-15 00:00:00+05:30   
0   2015-01-07 00:00:00+05:30    282.734890 2015-01-15 00:00:00+05:30   
..                        ...           ...                       ...   
499 2022-12-06 00:00:00+05:30    260.000000 2022-12-13 00:00:00+05:30   
584 2022-12-21 00:00:00+05:30    268.049988 2022-12-29 00:00:00+05:30   
566 2022-12-26 00:00:00+05:30     73.631806 2022-12-30 00:00:00+05:30   
475 2022-12-26 00:00:00+05:30    574.000000 2022-12-30 00:00:00+05:30   
596 2022-12-26 00:00:00+05:30    727.549988 2022-12-30 00:00:00+05:30   

     Selling_price          Stock  Return_rate  Holding_period  \
0       518.031555  TATAMOTORS.NS     5.925177             4.0   
0       121.026090        GSPL.NS    15.261766             4.0   
0        91.635414   JSWENERGY.BO     8.809888             9.0   
3       527.080811  TATAMOTORS.NS     7.917390             8.0   
0       301.866608        SBIN.NS     6.766663             8.0   
..             ...            ...          ...             ...   
499     274.500000        GSPL.NS     5.576923             7.0   
584     286.649994   JSWENERGY.BO     6.939007             8.0   
566      81.587990        SAIL.BO    10.805363             4.0   
475     615.250000        SBIN.NS     7.186411             4.0   
596     773.000000    JSWSTEEL.NS     6.246995             4.0   

     Return_rate_per_day  Days_diff  
0               1.481294        1.0  
0               3.815441        1.0  
0               0.978876        1.0  
3               0.989674        2.0  
0               0.845833        1.0  
..                   ...        ...  
499             0.796703        8.0  
584             0.867376       50.0  
566             2.701341       21.0  
475             1.796603       47.0  
596             1.561749       21.0  

[1169 rows x 9 columns]

In [5]:
stocks_df.drop_duplicates(subset=['Selling_date'],inplace=True)
stocks_df.drop_duplicates(subset=['Buying_date'],inplace=True)
stocks_df

Buying_date  Buying_price              Selling_date  \
0   2015-01-01 00:00:00+05:30    489.054225 2015-01-05 00:00:00+05:30   
0   2015-01-05 00:00:00+05:30    105.001072 2015-01-09 00:00:00+05:30   
0   2015-01-06 00:00:00+05:30     84.216072 2015-01-15 00:00:00+05:30   
0   2015-01-07 00:00:00+05:30    142.191345 2015-01-19 00:00:00+05:30   
0   2015-01-14 00:00:00+05:30    393.062103 2015-01-21 00:00:00+05:30   
..                        ...           ...                       ...   
497 2022-11-21 00:00:00+05:30    235.199997 2022-11-28 00:00:00+05:30   
419 2022-11-22 00:00:00+05:30   2545.000000 2022-11-29 00:00:00+05:30   
499 2022-12-06 00:00:00+05:30    260.000000 2022-12-13 00:00:00+05:30   
584 2022-12-21 00:00:00+05:30    268.049988 2022-12-29 00:00:00+05:30   
566 2022-12-26 00:00:00+05:30     73.631806 2022-12-30 00:00:00+05:30   

     Selling_price          Stock  Return_rate  Holding_period  \
0       518.031555  TATAMOTORS.NS     5.925177             4.0   
0       121.026090        GSPL.NS    15.261766             4.0   
0        91.635414   JSWENERGY.BO     8.809888             9.0   
0       154.883148     YESBANK.NS     8.925862            12.0   
0       426.862762    RELIANCE.NS     8.599318             7.0   
..             ...            ...          ...             ...   
497     279.000000        GSPL.NS    18.622451             7.0   
419    2712.199951    RELIANCE.NS     6.569743             7.0   
499     274.500000        GSPL.NS     5.576923             7.0   
584     286.649994   JSWENERGY.BO     6.939007             8.0   
566      81.587990        SAIL.BO    10.805363             4.0   

     Return_rate_per_day  Days_diff  
0               1.481294        1.0  
0               3.815441        1.0  
0               0.978876        1.0  
0               0.743822        1.0  
0               1.228474        1.0  
..                   ...        ...  
497             2.660350        5.0  
419             0.938535       13.0  
499             0.796703        8.0  
584             0.867376       50.0  
566             2.701341       21.0  

[569 rows x 9 columns]

In [6]:
stocks_df['Days_diff'] = (stocks_df['Buying_date'] - stocks_df['Selling_date'].shift(1)).dt.days
stocks_df = stocks_df.fillna(1)
stocks_df = stocks_df[stocks_df['Days_diff']>=0]
stocks_df

Buying_date  Buying_price              Selling_date  \
0   2015-01-01 00:00:00+05:30    489.054225 2015-01-05 00:00:00+05:30   
0   2015-01-05 00:00:00+05:30    105.001072 2015-01-09 00:00:00+05:30   
3   2015-01-22 00:00:00+05:30    108.960747 2015-01-28 00:00:00+05:30   
4   2015-02-02 00:00:00+05:30    108.029050 2015-02-05 00:00:00+05:30   
8   2015-02-09 00:00:00+05:30   2818.359678 2015-02-13 00:00:00+05:30   
..                        ...           ...                       ...   
434 2022-09-26 00:00:00+05:30    755.049988 2022-10-04 00:00:00+05:30   
409 2022-10-12 00:00:00+05:30   2352.000000 2022-10-19 00:00:00+05:30   
497 2022-11-21 00:00:00+05:30    235.199997 2022-11-28 00:00:00+05:30   
499 2022-12-06 00:00:00+05:30    260.000000 2022-12-13 00:00:00+05:30   
584 2022-12-21 00:00:00+05:30    268.049988 2022-12-29 00:00:00+05:30   

     Selling_price          Stock  Return_rate  Holding_period  \
0       518.031555  TATAMOTORS.NS     5.925177             4.0   
0       121.026090        GSPL.NS    15.261766             4.0   
3       115.156472        GSPL.NS     5.686199             6.0   
4       113.758929        GSPL.NS     5.304017             3.0   
8      3122.217922     DRREDDY.NS    10.781386             4.0   
..             ...            ...          ...             ...   
434     813.000000  BHARTIARTL.NS     7.674990             8.0   
409    2493.899902    RELIANCE.NS     6.033159             7.0   
497     279.000000        GSPL.NS    18.622451             7.0   
499     274.500000        GSPL.NS     5.576923             7.0   
584     286.649994   JSWENERGY.BO     6.939007             8.0   

     Return_rate_per_day  Days_diff  
0               1.481294        1.0  
0               3.815441        0.0  
3               0.947700        1.0  
4               1.768006        5.0  
8               2.695347        4.0  
..                   ...        ...  
434             0.959374        7.0  
409             0.861880        6.0  
497             2.660350        6.0  
499             0.796703        7.0  
584             0.867376        8.0  

[225 rows x 9 columns]

In [7]:
stocks_df['Stock'].value_counts()

YESBANK.NS       30
JSWENERGY.BO     25
TATAMOTORS.NS    24
GSPL.NS          24
JSWSTEEL.NS      21
BHARTIARTL.NS    21
LAURUSLABS.NS    20
SAIL.BO          18
SBIN.NS          15
RELIANCE.NS      12
DRREDDY.NS        8
NTPC.NS           7
Name: Stock, dtype: int64

In [10]:
investable_money = 100000
transactions = []

for inv in range(stocks_df.shape[0]):

    total_unit = investable_money // stocks_df.iloc[inv]['Buying_price']
    transaction_price = stocks_df.iloc[inv]['Buying_price']
    
    investable_money -= total_unit * transaction_price

    # Buy transaction
    transactions.append({
        'Date': stocks_df.iloc[inv]['Buying_date'],
        'Stock': stocks_df.iloc[inv]['Stock'],
        'Transaction_price': transaction_price,
        'Event': 'Buy',
        'Total_unit': total_unit,
        'Wallet_amount':"{:.2f}".format(investable_money)
    })

    investable_money += total_unit * stocks_df.iloc[inv]['Selling_price'] - (total_unit * stocks_df.iloc[inv]['Selling_price']) * 0.05

    # Sell transaction
    transactions.append({
        'Date': stocks_df.iloc[inv]['Selling_date'],
        'Stock': stocks_df.iloc[inv]['Stock'],
        'Transaction_price': stocks_df.iloc[inv]['Selling_price'],
        'Event': 'Sell',
        'Total_unit': total_unit,
        'Wallet_amount':"{:.2f}".format(investable_money)
    })

# Create a new dataframe with the transactions
transactions_df = pd.DataFrame(transactions)

# Write the dataframe to a CSV file
transactions_df.to_csv('transactions.csv', index=False)
transactions_df

Date          Stock  Transaction_price Event  \
0   2015-01-01 00:00:00+05:30  TATAMOTORS.NS         489.054225   Buy   
1   2015-01-05 00:00:00+05:30  TATAMOTORS.NS         518.031555  Sell   
2   2015-01-05 00:00:00+05:30        GSPL.NS         105.001072   Buy   
3   2015-01-09 00:00:00+05:30        GSPL.NS         121.026090  Sell   
4   2015-01-22 00:00:00+05:30        GSPL.NS         108.960747   Buy   
..                        ...            ...                ...   ...   
445 2022-11-28 00:00:00+05:30        GSPL.NS         279.000000  Sell   
446 2022-12-06 00:00:00+05:30        GSPL.NS         260.000000   Buy   
447 2022-12-13 00:00:00+05:30        GSPL.NS         274.500000  Sell   
448 2022-12-21 00:00:00+05:30   JSWENERGY.BO         268.049988   Buy   
449 2022-12-29 00:00:00+05:30   JSWENERGY.BO         286.649994  Sell   

     Total_unit Wallet_amount  
0         204.0        232.94  
1         204.0     100627.45  
2         958.0         36.43  
3         958.0     110182.27  
4        1011.0         22.96  
..          ...           ...  
445    943268.0  250013398.94  
446    961589.0        258.94  
447    961589.0  250758630.41  
448    935492.0         11.23  
449    935492.0  254750848.52  

[450 rows x 6 columns]

In [9]:
print(f"Total Money made, {round(float(transactions_df.loc[len(transactions_df)-1]['Wallet_amount'])/1000000,2)} Cr.")

Total Money made, 254.75 Cr.
